In [20]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='oAnxta94LlKVCPfY0LQ66sVpxiDW8-SAeR93BjCQhGql',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-gb.cloud-object-storage.appdomain.cloud')

bucket = 'nutritionaiagent-donotdelete-pr-kiiibiqntitp7v'
object_key = 'sample_nutrition_data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,Food Item,Calories,Tags,Suitable For
0,Millet Upma,210,"vegetarian, gluten-free",weight_loss
1,Grilled Chicken,250,"non-veg, high_protein",muscle_gain
2,Paneer Salad,280,"vegetarian, dairy",muscle_gain
3,Brown Rice + Dal,300,"vegetarian, wholegrain",general_health
4,Veg Salad,180,"vegetarian, low_calorie",weight_loss


In [21]:
# User input simulation
user_goal = "weight_loss"
user_allergies = ["dairy"]
user_preferences = ["vegetarian"]

# Filter meals based on goal
filtered = df_1[df_1["Suitable For"] == user_goal]

# Remove foods with allergens
def is_safe(tags, allergies):
    return all(a.lower() not in tags.lower() for a in allergies)

filtered = filtered[filtered["Tags"].apply(lambda x: is_safe(x, user_allergies))]

# Filter based on preference
def match_preferences(tags, prefs):
    return any(p.lower() in tags.lower() for p in prefs)

filtered = filtered[filtered["Tags"].apply(lambda x: match_preferences(x, user_preferences))]

# Show result
filtered.reset_index(drop=True, inplace=True)
filtered


,Food Item,Calories,Tags,Suitable For
0,Millet Upma,210,"vegetarian, gluten-free",weight_loss
1,Veg Salad,180,"vegetarian, low_calorie",weight_loss


In [ ]:
# Install transformers (only once needed)
!pip install transformers --quiet

# Hide tokenizer warning
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
# Example goal: "weight_loss"
user_goal = "weight_loss"

# Use df_1 because that's the name of your dataset loaded from IBM Cloud
filtered = df_1[df_1['Suitable For'].str.contains(user_goal, case=False)]
filtered.reset_index(drop=True, inplace=True)

# Show the result to verify
filtered.head()
